# Import Libraries

In [25]:
import os
import pickle
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Set Parameters

In [26]:
dataset_nr = 1
assert dataset_nr == 1 or 2

embedding_method = 'e2v'
assert embedding_method == 'e2v' or 'm2v'

NO_GENE_PRODUCTS = 'no gene products'
SINGLE_RELATION_TYPE = 'only one relation type'
concept_changes = None  # set to None when no conceptual changes are wanted
    
if concept_changes == NO_GENE_PRODUCTS:
    suffix = '_nogeneprods'
elif concept_changes == SINGLE_RELATION_TYPE:
    suffix = '_singlerel'
else:
    suffix = ''

seeded_emb = False

if seeded_emb:
    fixed_emb = '_seeded'
else:
    fixed_emb = ''
    
if seeded_emb:
    title_seeded = ' with fixed node embeddings'
else:
    title_seeded = ''

# Get Result Paths

In [27]:
curr_working_dir = os.getcwd()
curr_output_dir = os.path.join(curr_working_dir, 'output')
dataset_output_dir = os.path.join(curr_output_dir, f'g{dataset_nr}_{embedding_method}{fixed_emb}{suffix}')

if not os.path.exists(dataset_output_dir):
    print('First, run the edge2vec embedding script. Then, run this script.')
else:
    print(f'Output folder for dataset {dataset_nr} exists and will be loaded: {dataset_output_dir}')
    
run_folders_list = []
for item in os.listdir(dataset_output_dir):
    curr_path = os.path.join(dataset_output_dir, item)
    if os.path.isdir(curr_path) and 'run' in item:
        run_folders_list.append(item)

print(f'A total of {len(run_folders_list)} runs will be included in the analysis.')

run_folders_paths = []
pred_folders_paths = []
for run_folder in run_folders_list:
    run_path = os.path.join(dataset_output_dir, run_folder)
    run_folders_paths.append(run_path)
    pred_run_path = os.path.join(run_path, 'pred')
    pred_folders_paths.append(pred_run_path)
    print(pred_run_path)

Output folder for dataset 1 exists and will be loaded: c:\Users\rosa-\Google Drive\Msc_Bioinformatics\thesis\XAIFO-ThesisProject\output\g1_e2v
A total of 10 runs will be included in the analysis.
c:\Users\rosa-\Google Drive\Msc_Bioinformatics\thesis\XAIFO-ThesisProject\output\g1_e2v\run_001\pred
c:\Users\rosa-\Google Drive\Msc_Bioinformatics\thesis\XAIFO-ThesisProject\output\g1_e2v\run_002\pred
c:\Users\rosa-\Google Drive\Msc_Bioinformatics\thesis\XAIFO-ThesisProject\output\g1_e2v\run_003\pred
c:\Users\rosa-\Google Drive\Msc_Bioinformatics\thesis\XAIFO-ThesisProject\output\g1_e2v\run_004\pred
c:\Users\rosa-\Google Drive\Msc_Bioinformatics\thesis\XAIFO-ThesisProject\output\g1_e2v\run_005\pred
c:\Users\rosa-\Google Drive\Msc_Bioinformatics\thesis\XAIFO-ThesisProject\output\g1_e2v\run_006\pred
c:\Users\rosa-\Google Drive\Msc_Bioinformatics\thesis\XAIFO-ThesisProject\output\g1_e2v\run_007\pred
c:\Users\rosa-\Google Drive\Msc_Bioinformatics\thesis\XAIFO-ThesisProject\output\g1_e2v\run_008\p

In [28]:
auc_scores_all_runs = []
auc_loss_scores_all_runs = []

for run_name, pred_folder in zip(run_folders_list, pred_folders_paths):
    with open(f'{pred_folder}/performance_scores_{dataset_nr}_{embedding_method}{suffix}.pkl', 'rb') as f:
        loaded_info = pickle.load(f)
        
    keys = ['AUC Train', 'AUC Validation', 'AUC Test']
    for key in keys:
        auc_scores = loaded_info[key]
        for index, auc_score in enumerate(auc_scores):
            auc_scores_per_run = {'run': run_name, 'name': key, 'iteration': index, 'score': auc_score}
            auc_scores_all_runs.append(auc_scores_per_run)
            auc_loss_scores_all_runs.append(auc_scores_per_run)

    loss_scores = loaded_info['Loss']
    for index, loss_score in enumerate(loss_scores):
        formatted_loss_score = float(np.log10(loss_score))
        loss_scores_per_run = {'run': run_name, 'name': 'Cross-Entropy Loss', 'iteration': index, 'score': formatted_loss_score}
        auc_loss_scores_all_runs.append(loss_scores_per_run)

# Plot ROC Curves, AUC-ROC Scores and F1 Scores for Each Model

## ROC Curves

In [29]:
all_settings = [
    {'embedding_method': 'e2v',
     'fixed_emb': '',
     'dataset_nr': 1},
    {'embedding_method': 'e2v',
     'fixed_emb': '',
     'dataset_nr': 2}
]

curr_working_dir = os.getcwd()
curr_output_dir = os.path.join(curr_working_dir, 'output')

final_test_auc_roc_scores_all_runs_all_models = []
roc_curve_all_runs_all_models = []
f1_scores_all_runs_all_models = []

for setting in all_settings:
    curr_dataset_output_dir = os.path.join(curr_output_dir, f'g{setting["dataset_nr"]}_{setting["embedding_method"]}{setting["fixed_emb"]}')

    if not os.path.exists(curr_dataset_output_dir):
        print('First, run the edge2vec embedding script. Then, run this script.')
    else:
        print(f'Output folder for dataset {dataset_nr} exists and will be loaded: {curr_dataset_output_dir}')
            
        run_folders_list = []
        for item in os.listdir(curr_dataset_output_dir):
            curr_path = os.path.join(curr_dataset_output_dir, item)
            if os.path.isdir(curr_path) and 'run' in item:
                run_folders_list.append(item)

        print(f'A total of {len(run_folders_list)} runs will be included in the analysis.')

        run_folders_paths = []
        curr_pred_folders_paths = []
        for run_folder in run_folders_list:
            run_path = os.path.join(curr_dataset_output_dir, run_folder)
            run_folders_paths.append(run_path)
            pred_run_path = os.path.join(run_path, 'pred')
            curr_pred_folders_paths.append(pred_run_path)
            
        for run_name, pred_folder in zip(run_folders_list, curr_pred_folders_paths):
            with open(f'{pred_folder}/performance_scores_{setting["dataset_nr"]}_{setting["embedding_method"]}.pkl', 'rb') as f:
                loaded_info = pickle.load(f)

            auc_roc_score = loaded_info['ROC AUC Score']
            formatted_auc_roc_score = float(auc_roc_score)
            auc_roc_score_per_run = {'Model': f'g{setting["dataset_nr"]}_{setting["embedding_method"]}{setting["fixed_emb"]}', 
                                        'ROC AUC Score': formatted_auc_roc_score}
            final_test_auc_roc_scores_all_runs_all_models.append(auc_roc_score_per_run)

            roc_fpr_scores = loaded_info['ROC FPR']
            roc_tpr_scores = loaded_info['ROC TPR']
            
            for fpr, tpr in zip(roc_fpr_scores, roc_tpr_scores):
                auc_per_threshold_per_run = {'Model': f'g{setting["dataset_nr"]}_{setting["embedding_method"]}{setting["fixed_emb"]}',
                                             'ROC FPR': fpr, 'ROC TPR': tpr}
                roc_curve_all_runs_all_models.append(auc_per_threshold_per_run)

            f1_score = loaded_info['F1 Score']
            formatted_f1_score = float(f1_score)
            f1_score_per_run = {'Model': f'g{setting["dataset_nr"]}_{setting["embedding_method"]}{setting["fixed_emb"]}', 
                                'F1 Score': formatted_f1_score}
            f1_scores_all_runs_all_models.append(f1_score_per_run)

Output folder for dataset 1 exists and will be loaded: c:\Users\rosa-\Google Drive\Msc_Bioinformatics\thesis\XAIFO-ThesisProject\output\g1_e2v
A total of 10 runs will be included in the analysis.
Output folder for dataset 1 exists and will be loaded: c:\Users\rosa-\Google Drive\Msc_Bioinformatics\thesis\XAIFO-ThesisProject\output\g2_e2v
A total of 10 runs will be included in the analysis.


In [30]:
roc_curve_all_runs_all_models = pd.DataFrame(roc_curve_all_runs_all_models)
roc_curve_all_runs_all_models

,Model,ROC FPR,ROC TPR
0,g1_e2v,0.000000,0.000000
1,g1_e2v,0.000000,0.000192
2,g1_e2v,0.000000,0.006531
3,g1_e2v,0.000000,0.006915
4,g1_e2v,0.000000,0.010949
...,...,...,...
36427,g2_e2v,0.875091,0.999637
36428,g2_e2v,0.875091,0.999818
36429,g2_e2v,0.934502,0.999818
36430,g2_e2v,0.934502,1.000000


In [31]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.set_title(f'ROC TPR/ROC FPR Distribution over Each Run for Each Model')
sns.scatterplot(data=roc_curve_all_runs_all_models, x="ROC FPR", y="ROC TPR", hue="Model", s=1)
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.legend(markerscale=10)

fig.savefig(f'{curr_output_dir}/roc_curves.png', bbox_inches='tight')
fig.clear()

<Figure size 800x600 with 0 Axes>

## F1 Scores

In [32]:
f1_scores_all_runs_all_models = pd.DataFrame(f1_scores_all_runs_all_models)
f1_scores_all_runs_all_models

,Model,F1 Score
0,g1_e2v,0.923960
1,g1_e2v,0.918695
2,g1_e2v,0.921543
3,g1_e2v,0.926259
4,g1_e2v,0.924300
5,g1_e2v,0.925021
6,g1_e2v,0.923388
7,g1_e2v,0.926945
8,g1_e2v,0.920598
9,g1_e2v,0.926584


In [33]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.set_title(f'F1 Scores Overview for Each Model')
sns.barplot(f1_scores_all_runs_all_models, x="Model", y="F1 Score", errorbar="sd", color='cornflowerblue')
ax.bar_label(ax.containers[0], fontsize=10)
ax.set_xlabel('Model Variant')
ax.set_ylabel('F1 Score')

fig.savefig(f'{curr_output_dir}/f1_scores.png', bbox_inches='tight')
fig.clear()

<Figure size 800x600 with 0 Axes>

## Final AUC-ROC Scores

In [34]:
final_test_auc_roc_scores_all_runs_all_models = pd.DataFrame(final_test_auc_roc_scores_all_runs_all_models)
final_test_auc_roc_scores_all_runs_all_models

,Model,ROC AUC Score
0,g1_e2v,0.981466
1,g1_e2v,0.978083
2,g1_e2v,0.977793
3,g1_e2v,0.979621
4,g1_e2v,0.977190
5,g1_e2v,0.978637
6,g1_e2v,0.976998
7,g1_e2v,0.979838
8,g1_e2v,0.976336
9,g1_e2v,0.978401


In [35]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.set_title(f'AUC-ROC Scores Overview for Each Model')
sns.barplot(final_test_auc_roc_scores_all_runs_all_models, x="Model", y="ROC AUC Score", errorbar="sd")
ax.bar_label(ax.containers[0], fontsize=10)
ax.set_xlabel('Model Variant')
ax.set_ylabel('AUC-ROC Score')

fig.savefig(f'{curr_output_dir}/auc_roc_scores.png', bbox_inches='tight')
fig.clear()

<Figure size 800x600 with 0 Axes>

# Plot Training Curve

In [36]:
auc_scores_all_runs = pd.DataFrame(auc_scores_all_runs)
auc_scores_all_runs

,run,name,iteration,score
0,run_001,AUC Train,0,0.938449
1,run_001,AUC Train,1,0.924796
2,run_001,AUC Train,2,0.902691
3,run_001,AUC Train,3,0.931373
4,run_001,AUC Train,4,0.950517
...,...,...,...,...
4465,run_010,AUC Test,144,0.978238
4466,run_010,AUC Test,145,0.978276
4467,run_010,AUC Test,146,0.978309
4468,run_010,AUC Test,147,0.978363


In [37]:
auc_loss_scores_all_runs = pd.DataFrame(auc_loss_scores_all_runs)
auc_loss_scores_all_runs

,run,name,iteration,score
0,run_001,AUC Train,0,0.938449
1,run_001,AUC Train,1,0.924796
2,run_001,AUC Train,2,0.902691
3,run_001,AUC Train,3,0.931373
4,run_001,AUC Train,4,0.950517
...,...,...,...,...
5955,run_010,Cross-Entropy Loss,144,-0.401013
5956,run_010,Cross-Entropy Loss,145,-0.403343
5957,run_010,Cross-Entropy Loss,146,-0.403418
5958,run_010,Cross-Entropy Loss,147,-0.403079


In [38]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.set_title(f'Training curve on dataset {dataset_nr} with method {embedding_method}{title_seeded} {concept_changes}')
sns.lineplot(data=auc_scores_all_runs, x='iteration', y='score', hue='name')
ax.set_ylim(0.85,1)
ax.set_xlabel('Iteration')
ax.set_ylabel('AUC-ROC Score')

print(dataset_output_dir)
fig.savefig(f'{dataset_output_dir}/training_curve.png', bbox_inches='tight')
fig.clear()

c:\Users\rosa-\Google Drive\Msc_Bioinformatics\thesis\XAIFO-ThesisProject\output\g1_e2v


<Figure size 800x800 with 0 Axes>

In [39]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.set_title(f'Training curve on dataset {dataset_nr} with method {embedding_method}{title_seeded} {concept_changes}')
sns.lineplot(data=auc_loss_scores_all_runs, x='iteration', y='score', hue='name')
ax.set_ylim(top=1)
ax.set_xlabel('Iteration')
ax.set_ylabel('AUC-ROC/log10(Loss)')

fig.savefig(f'{dataset_output_dir}/training_curve_with_loss.png', bbox_inches='tight')
fig.clear()

<Figure size 800x800 with 0 Axes>

# Similarity between top scoring symptom-drug pairs

In [40]:
drug_symptom_pairs_per_run = []

for index, pred_path in enumerate(pred_folders_paths):
    with open(f'{pred_path}/candidates_per_symptom_{dataset_nr}_{embedding_method}{suffix}.pkl', 'rb') as f:
        loaded_list = pickle.load(f)
        
        drug_symptom_pairs = []
        
        for _, row in loaded_list.iterrows():
            symptom_id = row['Symptom']
            candidates = row['Candidates']
            
            for candidate in candidates:
                drug_symptom_pairs.append(tuple([symptom_id, candidate]))
                
        total_drug_symptom_pairs = len(drug_symptom_pairs)
    
    drug_symptom_pairs_per_run.append(drug_symptom_pairs)

In [41]:
similarity_matrix = {}
ratios_non_diagonals = []

for index1, pairs1 in enumerate(drug_symptom_pairs_per_run):
    
    similarities = {}
    
    for index2, pairs2 in enumerate(drug_symptom_pairs_per_run):
        overlap = set([tuple(sorted(ele)) for ele in pairs1]) & set([tuple(sorted(ele)) for ele in pairs2])
        ratio_overlap = len(overlap) / total_drug_symptom_pairs * 100
        
        similarities[f'run {index2+1}'] = ratio_overlap
        
        if index1 != index2:
            ratios_non_diagonals.append(ratio_overlap)
        
    similarity_matrix[f'run {index1+1}'] = similarities
    
similarity_matrix_df = pd.DataFrame(similarity_matrix)
similarity_matrix_df

,run 1,run 2,run 3,run 4,run 5,run 6,run 7,run 8,run 9,run 10
run 1,100.000000,34.567901,51.851852,39.506173,29.629630,38.271605,28.395062,37.037037,54.320988,54.320988
run 2,34.567901,100.000000,32.098765,51.851852,53.086420,19.753086,62.962963,23.456790,23.456790,30.864198
run 3,51.851852,32.098765,100.000000,50.617284,33.333333,43.209877,27.160494,37.037037,51.851852,61.728395
run 4,39.506173,51.851852,50.617284,100.000000,51.851852,35.802469,53.086420,28.395062,43.209877,46.913580
run 5,29.629630,53.086420,33.333333,51.851852,100.000000,19.753086,44.444444,40.740741,24.691358,41.975309
run 6,38.271605,19.753086,43.209877,35.802469,19.753086,100.000000,27.160494,24.691358,48.148148,30.864198
run 7,28.395062,62.962963,27.160494,53.086420,44.444444,27.160494,100.000000,17.283951,32.098765,25.925926
run 8,37.037037,23.456790,37.037037,28.395062,40.740741,24.691358,17.283951,100.000000,33.333333,32.098765
run 9,54.320988,23.456790,51.851852,43.209877,24.691358,48.148148,32.098765,33.333333,100.000000,53.086420
run 10,54.320988,30.864198,61.728395,46.913580,41.975309,30.864198,25.925926,32.098765,53.086420,100.000000


In [42]:
mean_overlap_ratio = np.mean(ratios_non_diagonals)
median_overlap_ratio = np.median(ratios_non_diagonals)
print('Mean:', mean_overlap_ratio)
print('Median:', median_overlap_ratio)

Mean: 38.35390946502058
Median: 37.03703703703704


In [43]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.set_title(f'Overlap ratio (Mean: {mean_overlap_ratio}, Median: {median_overlap_ratio}) between list of predicted symptom-drug pairs per run on dataset {dataset_nr} with method {embedding_method}{title_seeded} {concept_changes}')
sns.heatmap(similarity_matrix_df, annot=True, fmt='.1f', linewidths=0.5, ax=ax, cmap='RdYlGn')
ax.collections[0].set_clim(0,100)

fig.savefig(f'{dataset_output_dir}/overlap_between_runs.png', bbox_inches='tight')
fig.clear()

<Figure size 800x800 with 0 Axes>

In [44]:
for i in range(0, len(drug_symptom_pairs_per_run)):
    if i == 0:
        overlapping_pairs_all_runs = set(drug_symptom_pairs_per_run[i])
    else:
        overlapping_pairs_all_runs = overlapping_pairs_all_runs & set(drug_symptom_pairs_per_run[i])
            
print(f'There are {len(overlapping_pairs_all_runs)} symptom-drug pairs that are found in the top list of drug candidates in {len(drug_symptom_pairs_per_run)} runs: \n {overlapping_pairs_all_runs}')

with open(f'{dataset_output_dir}/symptom_drug_pair_overlapping_all_runs_{dataset_nr}_{embedding_method}{fixed_emb}.pkl', 'wb') as f:
    pickle.dump(overlapping_pairs_all_runs, f)

There are 4 symptom-drug pairs that are found in the top list of drug candidates in 10 runs: 
 {('HP:0011675', '1576'), ('HP:0003115', '231'), ('HP:0001635', '231'), ('HP:0002650', '5345')}


In [45]:
same_drug_symptom_pairs_thresholded = set()

threshold = 0.7
total_runs = len(drug_symptom_pairs_per_run)
min_nr_runs = int(threshold * total_runs)

for i in range(0, len(drug_symptom_pairs_per_run)):
    for pair in drug_symptom_pairs_per_run[i]:
        same_pairs = 0
        for j in range(0, len(drug_symptom_pairs_per_run)):
            for pair_to_compare in drug_symptom_pairs_per_run[j]:
                if pair == pair_to_compare:
                    same_pairs += 1
                        
        #print(f'For pair {pair} from run {i}, there are {same_pairs} same pairs found in list of all runs.')
        if same_pairs >= min_nr_runs:
            same_drug_symptom_pairs_thresholded.add(pair)
            
print(f'There are {len(same_drug_symptom_pairs_thresholded)} symptom-drug pairs that are found in the top list of drug candidates in at least {min_nr_runs} of the {total_runs} runs: \n {same_drug_symptom_pairs_thresholded}')

with open(f'{dataset_output_dir}/symptom_drug_pair_overlapping_{min_nr_runs}_runs_{dataset_nr}_{embedding_method}{fixed_emb}{suffix}.pkl', 'wb') as f:
    pickle.dump(same_drug_symptom_pairs_thresholded, f)

There are 23 symptom-drug pairs that are found in the top list of drug candidates in at least 7 of the 10 runs: 
 {('HP:0100543', '5345'), ('HP:0003560', '231'), ('HP:0003707', '231'), ('HP:0011675', '522'), ('HP:0002650', '5345'), ('HP:0011675', '1576'), ('HP:0002515', '5345'), ('HP:0003323', '231'), ('HP:0001635', '231'), ('HP:0001270', '5345'), ('HP:0001263', '5345'), ('HP:0001290', '5252'), ('HP:0001290', '5345'), ('HP:0001265', '5345'), ('HP:0003236', '231'), ('HP:0001256', '5345'), ('HP:0001638', '1576'), ('HP:0001644', '1576'), ('HP:0002093', '5345'), ('HP:0000750', '5345'), ('HP:0001638', '231'), ('HP:0001371', '5345'), ('HP:0003115', '231')}


# Check overlap between each model

In [46]:
nodes = pd.read_csv(f'output/indexed_nodes_{dataset_nr}.csv')
nodes.drop('index_id', axis=1, inplace=True)
nodes['semantic'] = nodes['semantic'].astype('category')
nodes

,id,semantic,label,semantic_id
0,WormBase:WBGene00000389,ORTH,cdc-25.4,5
1,ZP:0018675,DISO,right side lateral plate mesoderm mislocalised...,1
2,ZFIN:ZDB-GENE-040426-1197,ORTH,tbc1d5,5
3,5,DRUG,(S)-nicardipine,2
4,RGD:3443,ORTH,Ptk2,5
...,...,...,...,...
10029,MP:0009763,DISO,increased sensitivity to induced morbidity/mor...,1
10030,MP:0011057,DISO,absent brain ependyma motile cilia,1
10031,MP:0001412,DISO,excessive scratching,1
10032,WBPhenotype:0004023,DISO,frequency of body bend variant,1


In [47]:
all_settings = [
    {'embedding_method': 'e2v',
     'fixed_emb': '',
     'dataset_nr': 1},
    {'embedding_method': 'e2v',
     'fixed_emb': '',
     'dataset_nr': 2}
]

curr_working_dir = os.getcwd()
curr_output_dir = os.path.join(curr_working_dir, 'output')

overlapping_pairs = []

for setting in all_settings:
    dataset_output_dir = os.path.join(curr_output_dir, f'g{setting["dataset_nr"]}_{setting["embedding_method"]}{setting["fixed_emb"]}')

    if not os.path.exists(dataset_output_dir):
        print('First, run the edge2vec embedding script. Then, run this script.')
    else:
        print(f'Output folder for dataset {dataset_nr} exists and will be loaded: {dataset_output_dir}')
        
        with open(f'{dataset_output_dir}/symptom_drug_pair_overlapping_all_runs_{setting["dataset_nr"]}_{setting["embedding_method"]}{setting["fixed_emb"]}.pkl', 'rb') as f:
            loaded_list = pickle.load(f)
            print(loaded_list)
            overlapping_pairs.append(loaded_list)

Output folder for dataset 1 exists and will be loaded: c:\Users\rosa-\Google Drive\Msc_Bioinformatics\thesis\XAIFO-ThesisProject\output\g1_e2v
{('HP:0011675', '1576'), ('HP:0001635', '231'), ('HP:0003115', '231'), ('HP:0002650', '5345')}
Output folder for dataset 1 exists and will be loaded: c:\Users\rosa-\Google Drive\Msc_Bioinformatics\thesis\XAIFO-ThesisProject\output\g2_e2v
{('HP:0001638', '1576'), ('HP:0001290', '269'), ('HP:0001644', '1576'), ('HP:0003115', '1576'), ('HP:0003236', '1576'), ('HP:0011675', '1576')}


In [48]:
dataset1_emb_overlap = overlapping_pairs[0].intersection(overlapping_pairs[1])
for pair in dataset1_emb_overlap:
    symptom_id, drug_id = pair
    symptom_name = nodes.loc[nodes['id'] == symptom_id]['label'].iloc[0]
    drug_name = nodes.loc[nodes['id'] == drug_id]['label'].iloc[0]
    print(drug_name, 'treats', symptom_name)

levosimendan treats Arrhythmia
